# Deconvolution with Rascil and Radler

This script builds a simulated dataset and deconvolves it with Rascil and Radler implementations of the multiscale algorithm.

In [ ]:
%matplotlib inline

import os
import sys

sys.path.append(os.path.join('..', '..'))

from matplotlib import pylab

pylab.rcParams['figure.figsize'] = (30.0, 30.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy

from astropy.coordinates import SkyCoord
from astropy import units as u

from matplotlib import pyplot as plt


from rascil.data_models import PolarisationFrame

from rascil.processing_components import create_blockvisibility, \
    deconvolve_cube, restore_cube, create_named_configuration, create_test_image, \
    create_image_from_visibility, advise_wide_field, invert_ng, predict_ng

import radler as rd


The following block creates a simulated dataset, using the same steps used in imaging.ipynb 

In [ ]:
# Construct LOW core configuration
lowr3 = create_named_configuration('LOWBD2', rmax=750.0)

# Create visibilitiesz
times = numpy.zeros([1])
frequency = numpy.array([1e8])
channel_bandwidth = numpy.array([1e6])
phasecentre = SkyCoord(ra=+15.0 * u.deg, dec=-45.0 * u.deg, frame='icrs', equinox='J2000')
vt = create_blockvisibility(lowr3, times, frequency, channel_bandwidth=channel_bandwidth,
                       weight=1.0, phasecentre=phasecentre, polarisation_frame=PolarisationFrame('stokesI'))

advice = advise_wide_field(vt, guard_band_image=3.0, delA=0.1, facets=1, wprojection_planes=1, 
                           oversampling_synthesised_beam=4.0)
cellsize = advice['cellsize']

#Read the venerable test image, constructing an image
m31image = create_test_image(frequency=frequency, cellsize=cellsize,
                             phasecentre=phasecentre)
nchan, npol, ny, nx = m31image["pixels"].data.shape

vt = predict_ng(vt, m31image, context='2d')

# Make the dirty image and point spread function
model = create_image_from_visibility(vt, cellsize=cellsize, npixel=512)
dirty, sumwt = invert_ng(vt, model, context='2d')
psf, sumwt = invert_ng(vt, model, context='2d', dopsf=True)


Define deconvolution settings for Radler and Rascil deconvolution

In [ ]:
# Rascil deconvolution settings
n_iterations = 500
clean_threshold=0.001
frac_threshold=0.001
window_shape_='quarter'
loop_gain=0.7
ms_scales= [0, 3, 10, 30] 

# Radler deconvolution settings
settings = rd.Settings()
settings.algorithm_type = rd.AlgorithmType.multiscale
settings.trimmed_image_width = dirty.pixels.shape[2]
settings.trimmed_image_height = dirty.pixels.shape[3]
settings.pixel_scale.x = cellsize
settings.pixel_scale.y = cellsize  
settings.minor_iteration_count = n_iterations
settings.threshold = clean_threshold
settings.minor_loop_gain = loop_gain
settings.stop_on_negative_components = False
settings.multiscale.scale_list = ms_scales



Deconvolve using RASCIL

In [ ]:
comp, residual = deconvolve_cube(dirty, psf, niter=n_iterations, threshold=clean_threshold, fractional_threshold=frac_threshold,
                                 window_shape=window_shape_, gain=loop_gain, scales=ms_scales)

restored = restore_cube(comp, psf, residual)


Deconvolve using RADLER

In [ ]:
psf_radler = psf.pixels.to_numpy().astype(numpy.float32).squeeze()
dirty_radler = dirty.pixels.to_numpy().astype(numpy.float32).squeeze()
restored_radler = numpy.zeros_like(dirty_radler) 

beam_size = 0.0

radler_object = rd.Radler(settings, psf_radler, dirty_radler, restored_radler,
                          beam_size, rd.Polarization.stokes_i)
reached_threshold = False
reached_threshold = radler_object.perform(reached_threshold, 0)


Compare output images

In [ ]:
residual_rascil = residual.pixels.to_numpy().astype(numpy.float32).squeeze()
restored_rascil = restored.pixels.to_numpy().astype(numpy.float32).squeeze()

residual_min = numpy.min([numpy.min(residual_rascil) , numpy.min(dirty_radler)])
residual_max = numpy.max([numpy.max(residual_rascil) , numpy.max(dirty_radler)])

fig, ax = plt.subplots(2,2)
ax[0,0].imshow(residual_rascil, vmin=residual_min, vmax=residual_max)
ax[0,0].title.set_text('RASCIL: residual')
ax[0,1].imshow(restored_rascil)
ax[0,1].title.set_text('RASCIL: restored')

ax[1,0].imshow(dirty_radler, vmin=residual_min, vmax=residual_max)
ax[1,0].title.set_text('RADLER: residual')
ax[1,1].imshow(restored_radler)
ax[1,1].title.set_text('RADLER: restored')
